# Lession 21 - (LeNet for MNIST)

In [41]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
# New
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
# from keras.optimizers import Adam
import matplotlib.pyplot as plt
from time import time
%matplotlib inline

In [42]:
MNIST = np.load('../data/MNIST_train_1000.npz')
images = MNIST['train_images']
labels = MNIST['train_labels']
print(images.shape)
print(labels.shape)

(1000, 28, 28)
(1000,)


Need to add extra dimension for B&W images to make it a 4D tensor

In [43]:
X = np.expand_dims(images, -1)
print(X.shape)
P = pd.get_dummies(pd.DataFrame(labels, columns=['digits'], dtype='category')).values
print(P.shape)


(1000, 28, 28, 1)
(1000, 2)


Does it need scaling?  No

In [44]:
X.max()

1.0

In [45]:
# build LeNet
model = Sequential()
def BuildLeNet(model, input_shape=(32, 32, 3), outputs=10):
    # 20 channels out, 5 x 5 kernel, keep size the same
    model.add(Conv2D(20, 5, padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    # pool_size = size of pooling operation, strides=2 - Downsample by 2 in each directions
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Conv2D(50, 5, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dense(2,   activation='softmax'))
    return model

LeNet = BuildLeNet(model, input_shape=(28, 28, 1), outputs=2)
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 28, 28, 20)        520       
_________________________________________________________________
activation_15 (Activation)   (None, 28, 28, 20)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 14, 14, 50)        25050     
_________________________________________________________________
activation_16 (Activation)   (None, 14, 14, 50)        0         
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 7, 7, 50)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 2450)              0         
__________

In [46]:
LeNet.compile(loss='categorical_crossentropy',
             optimizer='Adam',
             metrics=['accuracy'])
hist = LeNet.fit(X, P, epochs=100, validation_split=0.2, verbose=1,
                callbacks=[EarlyStopping(patience=3)])

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3897 - acc: 0.7963 - val_loss: 0.1581 - val_acc: 0.9450
Epoch 2/100
800/800 [==============================] - 1s 854us/step - loss: 0.0526 - acc: 0.9763 - val_loss: 0.1715 - val_acc: 0.9600
Epoch 3/100
800/800 [==============================] - 1s 879us/step - loss: 0.0547 - acc: 0.9825 - val_loss: 0.1422 - val_acc: 0.9550
Epoch 4/100
800/800 [==============================] - 1s 811us/step - loss: 0.0122 - acc: 0.9988 - val_loss: 0.1108 - val_acc: 0.9800
Epoch 5/100
800/800 [==============================] - 1s 802us/step - loss: 0.0047 - acc: 1.0000 - val_loss: 0.1229 - val_acc: 0.9750
Epoch 6/100
800/800 [==============================] - 1s 804us/step - loss: 0.0039 - acc: 0.9988 - val_loss: 0.1192 - val_acc: 0.9750
Epoch 7/100
800/800 [==============================] - 1s 882us/step - loss: 0.0017 - acc: 1.0000 - val_loss: 0.1311 - val_acc: 0.9750
